In [2]:
def bcrp_run(fid, data, tc):
    '''
    Best constant rebalanced portfolio
    '''

    n, m = data.shape #n:rows, m:cols #n:507 m:30
    cum_ret = 1
    cumprod_ret = np.ones((n, 1), np.float64) #507,1

    daily_ret = np.ones((n, 1), np.float64) #507,1

    day_weight = np.ones((m, 1), np.float64)/m  #30,1
    day_weight_o = np.zeros((m, 1), np.float64) # 30,1
    daily_portfolio = np.zeros((n,m)) #507,30

    #print log file head
    print(fid, '-------------------------------------\n')
    print(fid, 'Parameters [tc: %f].\n' % tc)
    print(fid, 'day\t Daily Return\t Total return\n')
    print(1, '-------------------------------------\n')

    '''
          if opts.quiet_mode == False:
            print(1, 'Parameters [tc: %f].\n' % tc)
            print(1, 'day\t Daily Return\t Total return\n')

    '''
    day_weight = opt_weights(data)
    
    

        #backtest
    for t in range(n):

            # Calculate t's portfolio at the beginning of t-th trading day
        # Normalize the constraint, always useless
        #print('day_weight is ', day_weight)
        day_weight = np.divide(day_weight, np.sum(day_weight, axis=0)) # elementwise divide and column-sum
        #print('day_weight after is ', day_weight)
        
        daily_portfolio[t,:] = day_weight.conj().T
        #print(daily_portfolio.shape)
            
        # Cal t's return and total return
        daily_ret[t, 0] = np.dot(data[t,:],day_weight)*(1-tc/2*np.sum(np.absolute(day_weight-day_weight_o) ))
        cum_ret = np.dot(cum_ret , daily_ret[t,0])
        cumprod_ret[t, 0] = cum_ret
        
        #Adjust weight[t, :] for the transaction cost issue

        day_weight_o = day_weight * data[t,:].conj().T / daily_ret[t, 0]

        print(fid, '%d\t%f\t%f\n' % (t, daily_ret[t,0], cumprod_ret[t,0]))
        #print(day_weight)
        #print('------------------------------------------------\n')
        #print(day_weight_o)

        '''
            if opts.quiet_mode == False:
                if np.mod(t, opts.display_interval) == 0:
                      print(1, '%d\t%f\t%f\n' % (t, daily_ret[:t,:1], cumprod_ret[:t,:1]))     
        '''

    
    print(fid, 'Market(tc=%.4f), Cumulative return: %.2f\n'% (tc, cum_ret))
    print(fid, '-------------------------------------\n')
    print(1, 'Market(tc=%.4f), Cumulative return: %.2f\n'% (tc, cum_ret))
    print(1, '-------------------------------------\n')

def opt_weights(X, max_leverage=1):
    x_0 = max_leverage * np.ones(X.shape[1]) / float(X.shape[1])
    objective = lambda b: -np.prod(X.dot(b))
    cons = ({'type': 'eq', 'fun': lambda b: max_leverage-np.sum(b)},)
    bnds = [(0., max_leverage)]*len(x_0)
    res = minimize(objective, x_0, bounds=bnds, constraints=cons, method='slsqp', options={'ftol': 1e-07})
    return res.x

from __future__ import print_function
import numpy as np
from scipy.optimize import minimize
import scipy.io as sio
import logging

logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG)

djia = sio.loadmat('djia.mat') # load a test input
bcrp_run(1, djia['data'], 0)
#opt_weights(djia['data'])

1 -------------------------------------

1 Parameters [tc: 0.000000].

1 day	 Daily Return	 Total return

1 -------------------------------------

1 0	0.992321	0.992321

1 1	1.020982	1.013142

1 2	1.004569	1.017771

1 3	1.024084	1.042282

1 4	1.003315	1.045738

1 5	1.010939	1.057177

1 6	0.978700	1.034659

1 7	0.997429	1.031999

1 8	0.991959	1.023700

1 9	1.032239	1.056704

1 10	1.048611	1.108071

1 11	0.989140	1.096038

1 12	1.002463	1.098737

1 13	1.019330	1.119976

1 14	0.992632	1.111724

1 15	1.015884	1.129383

1 16	0.992630	1.121059

1 17	1.010848	1.133220

1 18	0.986638	1.118079

1 19	1.021894	1.142558

1 20	0.987980	1.128824

1 21	1.010382	1.140544

1 22	1.003428	1.144454

1 23	1.024997	1.173061

1 24	0.978470	1.147805

1 25	0.974525	1.118565

1 26	1.002271	1.121105

1 27	0.990736	1.110719

1 28	0.996067	1.106350

1 29	1.002194	1.108777

1 30	0.991300	1.099131

1 31	0.993014	1.091453

1 32	1.009470	1.101789

1 33	0.986545	1.086964

1 34	1.008805	1.096535

1 35	1.010096	1.107605
